In [11]:
import sys
sys.path.insert(1, '/Users/chinmayshrivastava/Desktop/OSCA/V4.0/scraping/crawler')
from generalscraper import *

In [12]:
paragraphs = pd.DataFrame(columns=['databaseid', 'tag', 'text', 'links'])
headings = pd.DataFrame(columns=['databaseid', 'tag', 'headingtext', 'subheading', 'nextheading', 'paragraphs'])
database = pd.DataFrame(columns=['topicheadingID', 'url', 'totalsubheadings', 'totalparagraphs'])

In [13]:
kwdata = pd.read_pickle('synonymsdata.pickle')
urlscraped = {}
urlstoscrape = []
kwanalyzed = {}

In [19]:
# i=0
kws_ = 0
howmanydocuments = len(database.index)
for kwid in kwdata:
#     i+=1
    if kwid == 0:
        continue
    if kwdata[kwid]['name'] in kwanalyzed:
        continue
    if kws_==1:
        break
    kws_ += 1
    
    print('Analysing:', kwdata[kwid]['name'])
    q = f"what is {kwdata[kwid]['name']} blockchain"
    engine = Google()
    results = engine.search(q)
    links = results.links()
    
    clear_output(wait=True)
    
    if len(links)<10:
        time.sleep(120)
        continue
    
    kwanalyzed[kwdata[kwid]['name']] = len(links)
    
    i = 0
    
    for url in links:
        
        i+=1
        print((i*100)/len(links))
        
        host = get_host_and_path(url)[0]
        try:
            url = urlcheck(url, host)
        except:
            continue
        print('scraping', url)
        
        if url in urlscraped:
            continue
        try:
            if url[-3:] == 'pdf':
                urlscraped[url] = False
                continue
            document_, headings_, paragraphs_ = articlescraper(url, len(database.index), dynamic=True, timetofetch=0.5)
        except:
            urlscraped[url] = False
            continue
        
        if adddocument(headings_, paragraphs_):
            # append to the database
            database.loc[len(database.index)] = document_
            for element in headings_:
                headings.loc[len(headings.index)] = element
            for element in paragraphs_:
                paragraphs.loc[len(paragraphs.index)] = element
            howmanydocuments += 1
            print('document added successfully!')
            urlscraped[url] = True
        else:
            print('document was rejected!')
            urlscraped[url] = False
            continue
        
        for element in paragraphs_:
            if len(element[3])!=0:
                for subelement in element[3]:
#                     print(subelement)
                    if subelement[0] not in urlscraped:
                        urlstoscrape.append(subelement[0])
#                     clear_output(wait=True)

    clear_output(wait=True)
    
    print(howmanydocuments, "have been successfully scraped!")
    if howmanydocuments%10000==0:
        integer = howmanydocuments/10000
        with open(f'generaldatabase{integer}.pickle', 'wb') as file:
            pickle.dump(database, file)
        with open(f'generalheadings{integer}.pickle', 'wb') as file:
            pickle.dump(headings, file)
        with open(f'generalparagraphs{integer}.pickle', 'wb') as file:
            pickle.dump(paragraphs, file)
        paragraphs = pd.DataFrame(columns=['databaseid', 'tag', 'text', 'links'])
        headings = pd.DataFrame(columns=['databaseid', 'tag', 'headingtext', 'subheading', 'nextheading', 'paragraphs'])
        database = pd.DataFrame(columns=['topicheadingID', 'url', 'totalsubheadings', 'totalparagraphs'])
#     break
#     print(q, len(links), len(urlscraped), i)

2289 have been successfully scraped!


In [20]:
urlstoscrape

[]

In [26]:
paragraphs

,databaseid,tag,text,links
0,0,p,We bring you the best in blockchain developmen...,[]
1,0,p,"At 10Clouds, we have more than 12 years of exp...",[]
2,0,p,We're a Blockchain development company with ov...,[]
3,0,p,Our team has worked with a vast range of block...,[]
4,0,p,Need Blockchain developers quickly? We can hel...,[]
...,...,...,...,...
48017,2287,p,Using automated trading systems comes with dra...,[]
48018,2287,p,"Advance your career in investment banking, pri...",[]
48019,2287,p,to take your career to the next level! Learn s...,[]
48020,2287,uli,Share this article,[]


In [13]:
isEnglish("Port3 Network We're Hiring")

True

In [5]:
kwanalyzed

{'10clouds': 64}